In [1]:
!git clone https://github.com/neccam/slt.git

Cloning into 'slt'...
remote: Enumerating objects: 99, done.
remote: Total 99 (delta 0), reused 0 (delta 0), pack-reused 99
Unpacking objects: 100% (99/99), done.


In [2]:
%cd slt

/content/slt


In [3]:
!chmod +x data/download.sh
!data/download.sh

--2021-01-02 11:46:22--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.train
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1935289362 (1.8G)
Saving to: ‘phoenix14t.pami0.train’

phoenix14t.pami0.tr 100%[===================>]   1.80G  29.2MB/s    in 53s     

2021-01-02 11:47:15 (34.6 MB/s) - ‘phoenix14t.pami0.train’ saved [1935289362/1935289362]

--2021-01-02 11:47:15--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.dev
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 130831227 (125M)
Saving to: ‘phoenix14t.pami0.dev’

phoenix14t.pami0.de 100%[===================>] 124.77M  38.5MB/s    in 3.2s    

2021-01-02 11:47:19 (38.5 MB/s) - ‘phoenix14t.pami0.dev’ saved [1308312

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!cp ../drive/MyDrive/slt/requirements.txt requirements.txt

In [6]:
!pip -q install -r requirements.txt

     |████████████████████████████████| 112kB 5.7MB/s 
     |████████████████████████████████| 112kB 17.1MB/s 
     |████████████████████████████████| 163kB 20.2MB/s 
     |████████████████████████████████| 399kB 21.4MB/s 
     |████████████████████████████████| 2.3MB 24.4MB/s 
     |████████████████████████████████| 81kB 9.1MB/s 
     |████████████████████████████████| 61kB 7.9MB/s 
     |████████████████████████████████| 2.4MB 54.4MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 296kB 46.1MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 92kB 10.6MB/s 
     |████████████████████████████████| 92kB 10.9MB/s 
     |████████████████████████████████| 13.1MB 44.4MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 20.2MB 50.0MB/s 
     |█

In [7]:
!mkdir data/PHOENIX2014T 

!mv phoenix14t.pami0.dev data/PHOENIX2014T 
!mv phoenix14t.pami0.test data/PHOENIX2014T 
!mv phoenix14t.pami0.train data/PHOENIX2014T

!cp -r ../drive/MyDrive/slt/keypoints/dev_features.zip data/dev_features.zip
!cp -r ../drive/MyDrive/slt/keypoints/train_features.zip data/train_features.zip
!cp -r ../drive/MyDrive/slt/keypoints/test_features.zip data/test_features.zip

!unzip -q data/dev_features.zip -d data
!unzip -q data/train_features.zip -d data
!unzip -q data/test_features.zip -d data

In [8]:
from six.moves import cPickle
import gzip
import torch
import os
import numpy as np
from tqdm import tqdm
import pickle

wd='data'

def load_dataset_file(filename):
    with gzip.open(filename, "rb") as f:
        loaded_object = pickle.load(f)
        return loaded_object

for s in ['dev', 'test', 'train']:
  filename = "data/PHOENIX2014T/phoenix14t.pami0.{}".format(s)
  src_features = load_dataset_file(filename)

  for src in tqdm(src_features):
    name = os.path.basename(src['name']) + '.npy'

    dst = os.path.join(wd, os.path.join(s, name))
    
    new_features = np.load(dst)
    src['sign'] = torch.Tensor(new_features)

  with gzip.open(filename, "wb") as f:
    pickle.dump(src_features, f)

100%|██████████| 7096/7096 [00:32<00:00, 220.17it/s]


In [9]:
import yaml
with open("configs/sign.yaml") as f:
     list_doc = yaml.load(f)

list_doc['model']['model_dir'] = "hidden_feature"

list_doc['training']['recognition_loss_weight'] = 20


with open("configs/sign.yaml", "w") as f:
    yaml.dump(list_doc, f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
!python -m signjoey train configs/sign.yaml

2021-01-02 12:02:30.695254: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-01-02 12:02:36.795387: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-02 12:02:36.814895: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-01-02 12:02:36.861503: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-01-02 12:02:36.862409: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-01-02 12:02:36.862448: I tensorflow/stream_executor/platform/def

In [11]:
!cp -r ./sign_sample_model "../drive/MyDrive/slt/models/hidden_features" 